In [1]:
import numpy as np
from numba import cuda
import time
import matplotlib.pyplot as plt
from utils import significance_of_mean

### Synthetic data

Generating some random data.

In [2]:
np.random.seed(1)
A = np.asarray([np.random.randint(0,5,5) for _ in range(1)])
B = np.asarray([np.random.randint(0,5,5) for _ in range(1)])



In [3]:
x = list(A[0])
y = list(B[0])


$\bf{x}=[3, 4, 0, 1, 3]$, $\bf{y}=[0, 0, 1, 4, 4]$, and $\bf{z}=[0, 0, 0, 1, 1, 3, 3, 4, 4, 4]$ with sizes $m=5$, $n=5$, and $m+n=10$, respecitvely. The possible sums $s$ of a $m$-combination from $z$ ranges between $0\leq s \leq 18$.

# \# $j$-combinations s.t. their elements sum is equal to $s$.

Here is the exact algorithm outline in the article.

In [5]:
def getNumerator(x, y, dtype):
    x = [0] + x 
    m = len(x)
    n = len(y)
    z = x + y;z.sort()
    S = sum(z[m:])
    dtype = np.uint16
    

    N = np.zeros([S + 1, m ], dtype)
    N_old = N.copy()
    
    for i in range(1,(m+n)+1):
        for j in range(1,m +1):
            for s in range( S+1):
                if s==0 and j-1==0:
                    N[s,j-1] = 1
                elif i < j:
                    N[s,j-1] = 0
                elif j > 1 and z[i-1] <= s:
                    N[s,j-1] = N_old[s - z[i -1], j-2] + N_old[s,j-1]
                elif j > 1 and z[i-1] > s:
                    N[s,j-1] = N_old[s,j-1]
        
        N_old = N.copy()
        
    return N_old[:,-1]

In [6]:
Nsm = getNumerator(x,y, np.float64)

TypeError: 'float' object cannot be interpreted as an integer

From the calculated $N(s,m)$ the sought of $p$-value can be calculated: $P(s_{\text{obs}} \leq S |x, y)=\sum _{s=s_{obs}}^{\mathcal{S}}\frac{N(s,m)}{{m+n \choose m}}$

In [ ]:
def pValue(Numerator, sample):
    return np.round((Numerator / np.sum(Numerator))[sum(sample):].sum(), 3)

In [ ]:
pValue(Nsm, x)

The $p-value=0.464$

# Parallelization of algorithm

It is the two inner for-loops that can parallelized i.e., by keeping $i$ constant, $j$ and $s$ is paralellizible. Let's write those two loops in Numba.

In [ ]:
@cuda.jit("(u8[:,:], u8[:,:], u8, u8[:],u8)")
def fill_array_u4_v_u2(X1,X2, i_, z_,S):
    n = X1.shape[0]
    m = X1.shape[1]

    s, j = cuda.grid(2)
    
    if j >= m + 1 or s > S or j < 1:
        return
    
    if s==0 and int(j-1)==0:
        X2[s,j -1] = 1
    
    if i_ < j:
        X2[s, j - 1] = 0
        
    elif j>1 and z_[i_ - 1] <= s:
        X2[s, j - 1] = X1[s - int(z_[i_ - 1]), j - 2] + X1[s, j - 1]
    
    elif j>1 and z_[i_ - 1] > s:
        X2[s, j-1] = X1[s,j-1]
    

The getNumerator can now be rewritten with this new function to get it parallelized.

In [ ]:
def getNumeratorParallelized(A0, A1, dA0, dA1, m, n, S, dz, threadsperblock, blockspergrid, stream):
    for i in range(1, (m+n) + 1):
        fill_array_u4_v_u2[blockspergrid, threadsperblock, stream](dA0, dA1, np.uint64(i), dz, S)
        
        
        tmp = dA0
        dA0 = dA1
        dA1 = tmp
        
        
        
    dA0.to_host(stream)

    
    return A1[:,-1]

Here is some necessary initial step to use the GPU: Initiate array to write, and threadblocks and blockgrids on the GPU.

In [ ]:
def runParalellized(x, y):
    x = [0] + x 
    m = len(x)
    n = len(y)
    z = x + y;z.sort()
    S = sum(z[m:])
    dtype = np.uint64
    
    z = np.array(z, dtype)
    
    A0 = np.zeros([int(S) + 1, m], np.uint64)
    NN, NM = A0[:, :].shape
        
    threadsperblock = (64, 4)
    blockspergrid = (int(np.ceil((NN)/ threadsperblock[0])),
                     int(np.ceil(NM/threadsperblock[1] + 1))
                    )
            
    A1 = np.zeros([int(S) + 1, m], np.uint64)
    
    stream = cuda.stream()
    dz, dA0, dA1 = cuda.to_device(z, stream), cuda.to_device(A0, stream), cuda.to_device(A1, stream)
    
    return getNumeratorParallelized(A0,A1, dA0, dA1, m, n, S, dz, threadsperblock, blockspergrid, stream)
    

In [ ]:
NsmP = runParalellized(x,y)

Verify so they yield the same answer.

In [ ]:
np.allclose(NsmP, Nsm2)